# Step by step CDF creation

In [1]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pycdfpp
import requests
import xarray as xr

%matplotlib widget

## 1. Create an empty CD object

In [2]:
my_cdf = pycdfpp.CDF()
print(my_cdf)

CDF:
  version: 3.9.0
  majority: row
  compression: None

Attributes:

Variables:




## 2. Add some global attributes

In [3]:
my_cdf.add_attribute?

Docstring: add_attribute(self: pycdfpp._pycdfpp.CDF, name: str, values: List[Union[None, str, List[str], List[int], List[int], List[int], List[int], List[int], List[int], List[int], List[float], List[float], List[pycdfpp._pycdfpp.tt2000_t], List[pycdfpp._pycdfpp.epoch], List[pycdfpp._pycdfpp.epoch16], List[datetime.datetime]]]) -> pycdfpp._pycdfpp.Attribute
Type:      method

In [4]:
my_cdf.add_attribute("TITLE", ["My new CDF file"])

TITLE: "My new CDF file"

In [5]:
my_cdf.add_attribute("Creation date", [[datetime.now()]])

Creation date: [ [ [ 2023-09-04T16:17:03.494246
 ] ] ]

In [6]:
print(my_cdf)

CDF:
  version: 3.9.0
  majority: row
  compression: None

Attributes:
  TITLE: "My new CDF file"
  Creation date: [ [ [ 2023-09-04T16:17:03.494246
 ] ] ]

Variables:




## 3. Add some variables

In [7]:
my_cdf.add_variable?

Signature:
my_cdf.add_variable(
    name: str,
    values: numpy.ndarray = None,
    data_type=None,
    is_nrv: bool = False,
    compression: pycdfpp._pycdfpp.CompressionType = <CompressionType.no_compression: 0>,
    attributes: Mapping[str, List[Any]] = None,
)
Docstring: <no docstring>
File:      ~/PySide6env-p3.11/lib64/python3.11/site-packages/pycdfpp/__init__.py
Type:      method

### 3.1 Time related variables

In [8]:
my_cdf.add_variable(
    "Epoch",
    values=[
        datetime(2020, 1, 1, 12, 0, 5, microsecond=10000) + timedelta(seconds=i)
        for i in range(100)
    ],
    compression=pycdfpp.CompressionType.gzip_compression,
)

Epoch:
  shape: [ 100 ]
  type: CDF_TIME_TT2000
  record varry: True
  compression: GNU GZIP

  Attributes:

In [9]:
my_cdf.add_variable(
    "Epoch_from_dt64",
    values=np.arange(1e18, 11e17, 1e16, dtype=np.int64).astype("datetime64[ns]"),
    compression=pycdfpp.CompressionType.gzip_compression,
)

Epoch_from_dt64:
  shape: [ 10 ]
  type: CDF_TIME_TT2000
  record varry: True
  compression: GNU GZIP

  Attributes:

### 3.2 String variables such as [ISTP metada](https://spdf.gsfc.nasa.gov/istp_guide/variables.html#Metadata)

In [10]:
my_cdf.add_variable("Var1_labels", values=["Bx", "By", "Bz"], is_nrv=True)

Var1_labels:
  shape: [ 3, 2 ]
  type: CDF_CHAR
  record varry: Flase
  compression: None

  Attributes:

### 3.3 Regular data variables

In [11]:
my_cdf.add_variable(
    "Var1",
    values=np.ones((100, 3)),
    compression=pycdfpp.CompressionType.gzip_compression,
    attributes={"DEPEND_0": "Epoch"},
)

Var1:
  shape: [ 100, 3 ]
  type: CDF_DOUBLE
  record varry: True
  compression: GNU GZIP

  Attributes:
    DEPEND_0: "Epoch"

In [12]:
my_cdf.add_variable(
    "Some variable with many dimensions",
    values=np.ones((100, 3, 5, 6), dtype=np.int16),
    compression=pycdfpp.CompressionType.gzip_compression,
    data_type=pycdfpp.DataType.CDF_INT2,
    attributes={"DEPEND_0": "Epoch"},
)

Some variable with many dimensions:
  shape: [ 100, 3, 5, 6 ]
  type: CDF_INT2
  record varry: True
  compression: GNU GZIP

  Attributes:
    DEPEND_0: "Epoch"

### 3.4 Add attributes to variables

In [13]:
my_cdf["Var1"].add_attribute("Sum", [np.sum(my_cdf["Var1"])])

Sum: [ [ [ 300 ] ] ]

In [14]:
my_cdf["Var1"]

Var1:
  shape: [ 100, 3 ]
  type: CDF_DOUBLE
  record varry: True
  compression: GNU GZIP

  Attributes:
    DEPEND_0: "Epoch"
    Sum: [ [ [ 300 ] ] ]

In [15]:
my_cdf["Var1"].attributes["Sum"][0]

[300.0]

## 4. Save CDF object

In [16]:
pycdfpp.save(my_cdf, "/tmp/my_cdf.cdf")

True

In [17]:
!/usr/local/cdf/bin/cdfvalidate -debug /tmp/my_cdf.cdf

Validating "/tmp/my_cdf.cdf"...
  Checking CDR...@8
  Checking GDR...@320
  Checking CVVR...@1603
  Checking CVVR...@2529
  Checking VVR...@3044
  Checking CVVR...@3486
  Checking CVVR...@3994
  Checking ADR...@404
  Checking AEDR...@728
  Checking ADR...@799
  Checking AEDR...@1123
  Checking ADR...@4071
  Checking AEDR...@4395
  Checking AEDR...@4456
  Checking ADR...@4517
  Checking AEDR...@4841
  Checking VDR...@1187
  Checking VXR...@1531
  Checking CPR...@1575
  Checking VDR...@2113
  Checking VXR...@2457
  Checking CPR...@2501
  Checking VDR...@2656
  Checking VXR...@3000
  Checking VDR...@3062
  Checking VXR...@3414
  Checking CPR...@3458
  Checking VDR...@3554
  Checking VXR...@3922
  Checking CPR...@3966


In [18]:
bytes(pycdfpp.save(my_cdf))

b'\xcd\xf3\x00\x01\x00\x00\xff\xff\x00\x00\x00\x00\x00\x00\x018\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x01@\x00\x00\x00\x03\x00\x00\x00\x08\x00\x00\x00\x06\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\nCommon Data Format (CDF)\\nhttps://cdf.gsfc.nasa.gov\nSpace Physics Data Facility\nNASA/Goddard Space Flight Center\nGreenbelt, Maryland 20771 USA\n(User support: gsfc-cdf-support@lists.nasa.gov)\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00T\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\xa3\x00\x00\x00\x00\x00\x00\x01\x94\x00\x00\x00\x00\x00\x00\x13%\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0